In [1]:
import h2o
import pandas as pd

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_144"; Java(TM) SE Runtime Environment (build 1.8.0_144-b01); Java HotSpot(TM) 64-Bit Server VM (build 25.144-b01, mixed mode)
  Starting server from /usr/local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/y1/v_v3498n54g4rjytbs16kvwm0000gn/T/tmpq4w415c0
  JVM stdout: /var/folders/y1/v_v3498n54g4rjytbs16kvwm0000gn/T/tmpq4w415c0/h2o_bezawada_started_from_python.out
  JVM stderr: /var/folders/y1/v_v3498n54g4rjytbs16kvwm0000gn/T/tmpq4w415c0/h2o_bezawada_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Asia/Shanghai
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.4
H2O cluster version age:,1 month and 17 days
H2O cluster name:,H2O_from_python_bezawada_8tbegr
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [3]:
from h2o.automl import H2OAutoML

In [4]:
df = pd.read_json('/Users/bezawada/Desktop/know/nokia/swbot/swbot/analyzer/data/MNCombined/component/mn_combined_1Q2018_feature_components.json')

In [5]:
print(df.shape)
print (df.columns)

(7416, 75)
Index(['Internal Analysis Information', 'author', 'authorGroup', 'build',
       'description', 'faultAnalysisCustomerImpact', 'faultAnalysisFeature',
       'faultAnalysisId', 'faultAnalysisImpactToEndUser',
       'faultAnalysisImpactToOperator', 'faultAnalysisTitle', 'feature',
       'groupInCharge', 'problemReportId', 'reasonForTransfer', 'reportedDate',
       'severity', 'state', 'subsystem', 'title', 'feature_f', 'feature_title',
       'i_4GMAX (FDD)', 'i_ADMIN', 'i_BM', 'i_BTSOM', 'i_BTSSM', 'i_C-COMMON',
       'i_C-Plane', 'i_CC&SMCU', 'i_CCS-RT', 'i_CELLC', 'i_DCM L2',
       'i_DCM MED', 'i_DCM PHY', 'i_DCM Platform', 'i_DCMUM', 'i_DL PHY',
       'i_DSPHWAPI-RT', 'i_DSPi WMP', 'i_ENBC', 'i_FPGA', 'i_FTM', 'i_FZ AP',
       'i_FZ CI', 'i_FZ EI', 'i_FZ MF', 'i_FZ RT', 'i_FZ TR', 'i_HWAPI',
       'i_IMP', 'i_L1low', 'i_L2', 'i_LFS', 'i_LOM', 'i_LTEM2M3',
       'i_MAC DCM PS', 'i_MAC PS', 'i_MW DCM', 'i_PS CEVA', 'i_PS DSP',
       'i_Platform', 'i_RF', 'i_RF (f

In [6]:
dfs = df[['title', 'description', 'groupInCharge']]

In [7]:
dfs['info'] = df['title'] +  ' '  + df['description']
dfs = dfs[['info', 'groupInCharge']]

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
dfs = dfs.sample(frac=1).reset_index(drop=True)

In [9]:
dfs.head(2)

,info,groupInCharge
0,[TL18 Trunk] [FSIH] Counter M8012C89/ ACTIVE_T...,RABLTESWCBEOAM
1,[CIT][FL17A] UL throughput is lower than expec...,RABLTEWTS


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect_tfidf = TfidfVectorizer(encoding='utf-8',
                                 ngram_range=(1,2),
                                 max_features=5000)

### Below code uses RandomForest library for classification

In [15]:
dfs.columns

Index(['info', 'groupInCharge'], dtype='object')

In [16]:
hf = h2o.H2OFrame(dfs)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [20]:
print (type(hf))
print (hf.shape)
print (hf.columns)

<class 'h2o.frame.H2OFrame'>
(7467, 2)
['info', 'groupInCharge']


In [22]:
X_dtm = vect_tfidf.fit_transform(dfs['info'])

In [23]:
type(X_dtm)

scipy.sparse.csr.csr_matrix

In [24]:
X_dense = X_dtm.todense()

In [25]:
X_dense.shape

(7416, 5000)

In [26]:
tfidf_df = pd.DataFrame(X_dense)

In [28]:
tfidf_df['label'] = dfs['groupInCharge']

In [48]:
tfidf_df.columns

Index([      0,       1,       2,       3,       4,       5,       6,       7,
             8,       9,
       ...
           491,     492,     493,     494,     495,     496,     497,     498,
           499, 'label'],
      dtype='object', length=501)

In [49]:
tfidf_df.to_csv('tfidf.csv')

In [51]:
hf_df = h2o.import_file("tfidf.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [53]:
from h2o.estimators import H2ORandomForestEstimator

In [54]:
model = H2ORandomForestEstimator(ntrees=50, max_depth=20, nfolds=10)

In [80]:
hf_df_small = hf_df[1:100, :]

In [81]:
print (hf_df.shape)
print (hf_df_small.shape)

(7417, 502)
(99, 502)


In [89]:
training_columns = hf_df.columns[0:500]
label = 'C502'

In [90]:
model.train(x=training_columns, y=label, training_frame=hf_df_small)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [94]:
hf_df_test = hf_df[100:110, :]

In [95]:
performance = model.model_performance(test_data=hf_df_test)

In [96]:
print (performance)


ModelMetricsMultinomial: drf
** Reported on test data. **

MSE: 0.9319941109156009
RMSE: 0.9653984208168154
LogLoss: 15.927865385374867
Mean Per-Class Error: 0.022727272727272728
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



4G_RAN_ST_OAM1_HZ,4G_RAN_ST_RRM1_SH,4G_RAN_ST_TRS1_HZ,APsim Dev,BTS_SYSTEM_MAINT,EXT_LTE_ASB_CBE_GVE_CSV1,EXT_LTE_ASB_CBE_GVE_ST_OAM,EXT_LTE_ASB_CBE_MW_DCM,EXT_LTE_ASB_CBE_OAM,EXT_LTE_ASB_CBE_SISO,EXT_LTE_ASB_CNJ_OAM_FZ,EXT_LTE_ASB_CSH_CP,EXT_LTE_ASB_CSH_GVE_ST_KPI1,EXT_LTE_ASB_CSH_GVE_ST_RRM,EXT_LTE_ASB_CSH_L2,EXT_LTE_ASB_CSH_L2,EXT_LTE_ASB_CSH_PET,EXT_LTE_ASB_CSH_SISO,EXT_LTE_ASB_CSH_UPMACPS,EXT_LTE_ASB_CSH_UPTDDMACPS,EXT_LTE_ASB_CSH_UPTDDULPHY,EXT_RABLTECPSWPSNHD,EXT_RABLTEOAMSWPSNHD,EXT_RABLTESWPMC,EXT_RABLTETRXSWPSNHD,EXT_RPBROATLTE,EXT_RPESJUFXDD,FW2CA,FW2DA,FW2DC,FW2EHB,FW2FIB,FW2HA,FW2IRB,FW2PC,FZM_APP_GENERAL,FZM_OM_GENERAL,FZM_QT,FZM_SC_I&V_Triage,FZM_SE,FZM_SyVe,FZM_T&P,FZM_TRS_General,FZ_RT_ARM_Platform,FZ_RT_BM,FZ_RT_GENERAL,FZ_RT_Phy_Data,FZ_RT_cFZC_Platform,LTE_3PV_Fujitsu,LTE_AATNP2_UNP_GPL,LTE_CP_FZ_NANJING,LTE_CRT_FZ_NANJING,LTE_DEVBE1_CBE_L2,LTE_DEVBE1_CBE_SFE,LTE_DEVBE2_CBE_DCT,LTE_DEVBE3_CBE_LOMPM,LTE_DEVCAHZ_CHZ_UPMACPS,LTE_DEVCNJ_SC_MRO,LTE_DEVHZ1_CHZ_FV,LTE_DEVHZ1_CHZ_RFALU,LTE_DEVHZ2_CHZ_FV,LTE_DEVHZ2_CHZ_TEST,LTE_DEVHZ3_CHZ_FV,LTE_DEVHZ4_CHZ_PHY_5GMAX,LTE_DEVHZ5_CHZ_FV,LTE_DEVIR1_UIR_3PVALU,LTE_DEVIR1_UIR_SPECAIF,LTE_DEVKR1_PKR_MACPS,LTE_DEVLN1_FRLN_L2,LTE_DEVLN1_FRLN_MACPS,LTE_DEVLN1_FRLN_ULPHY,LTE_DEVMA1_PMA_DCMMWCP,LTE_DEVMA1_PMA_DCMMWCW,LTE_DEVMA1_PMA_DCMMWOAM,LTE_DEVMA1_PMA_DCMMWTOOLS,LTE_DEVMA1_PMA_DCMMWTRX,LTE_DEVMA1_PMA_OAMTEST,LTE_DEVMH1_UMH_RFDAS,LTE_DEVNP1_UNP_CP,LTE_DEVNP1_UNP_UP,LTE_DEVOU1_FOU_MACPS,LTE_DEVOU1_FOU_TEST,LTE_DEVPFHZ_CHZ_UPMACPS,LTE_DEVPS1_FRPS_CP,LTE_DEVPS1_FRPS_LF_CEVAPS,LTE_DEVSH1_CSH_UPET,LTE_DEVSH1_CSH_UPFDDULPHY,LTE_DEVUL1_GUL_CPTEST,LTE_DEVUL1_GUL_DLPHY_LF,LTE_DEVWR1_PWR_TEST,LTE_DEV_FZ1,LTE_DevIV_FZ_Krakow,LTE_Dev_FZ1,LTE_Dev_FZ12,LTE_Dev_FZ14_Nanjing,LTE_Dev_FZ15,LTE_Dev_FZ16_Nanjing,LTE_Dev_FZ2,LTE_Dev_FZ4_K2,LTE_Dev_FZ7,LTE_Dev_FZ9,LTE_FDD_SA_CP,LTE_FDD_SA_RRM_UP,LTE_FDD_SA_RRM_UP,LTE_FZM_3PV_OSS,LTE_FZM_3PV_TI,LTE_GRD_FOU_TEST1,LTE_GRD_FOU_TEST2,LTE_GRD_PWR_TEST2,LTE_GVE_COT_TP_BEJ,LTE_GVE_COT_TP_DAL,LTE_GVE_COT_TP_HZ,LTE_GVE_COT_TP_ULM,LTE_GVE_COT_TP_WRO,LTE_GVE_ET_SISO_BEJ,LTE_GVE_ET_SISO_DAL,LTE_GVE_ET_SISO_WRO,LTE_GVE_ET_ST_HFV1_HZ,LTE_GVE_ET_ST_HFV2_HZ,LTE_GVE_ET_ST_PET1_HZ,LTE_GVE_ET_ST_PET2_HZ,LTE_GVE_ET_ST_PET3_HZ,LTE_GVE_ET_ST_SISO1_HZ,LTE_GVE_ET_ST_SISO2_HZ,LTE_GVE_ET_ST_SISO3_HZ,LTE_GVE_NAMNV_ATT_NAP,LTE_GVE_NAMNV_SPRINT_NAP,LTE_GVE_NAMNV_VZW_NAP,LTE_GVE_ST_CP_ULM,LTE_GVE_ST_CSV1_SH,LTE_GVE_ST_CSV2_SH,LTE_GVE_ST_FIVE_HZ,LTE_GVE_ST_FeVe_LANNION,LTE_GVE_ST_NLV1_SH,LTE_GVE_ST_OM_ULM,LTE_GVE_ST_RFSOMCP_WRO,LTE_PET_FZ_KRAKOW,LTE_PET_FZ_NJ,LTE_SA_ANA,LTE_SA_Cap,LTE_SA_OMBASIC,LTE_SA_OMRNL,LTE_SA_PM,LTE_SA_TR,LTE_SCM_FZ,LTE_SyVe_FZ_Krakow,LTE_TDD_RRM_EI,LTE_TDD_SA_RRM_UP,LTE_TRS_FZ_NANJING,LTE_TRS_I&V_BLR1,LTE_VER_CSV_ST_SFE_WRO,LTE_VER_CSV_ST_UUF_WRO,LTE_VER_LAS_HZ1,LTE_VER_LAS_SH1,LTE_VER_LAS_SH4,LTE_VER_ST_CP_ULM,LTE_VER_ST_OM_ULM,LTE_VER_ST_OM_WRO,LTE_VER_ST_RRMIoT_WRO,LTE_VER_ST_RRM_LANNION,LTE_VER_ST_TT_ULM,LTM BLR System Verification,LTM General,NCIR_SW,NDCS_FW_Quanta,NIAHSDK,NIAHSRF,NIBESDCT,NIBESDCT,NIHZHRF_TDTECH,NIHZHWIVRFCN11,NIHZHWIVRFCN1L1,NIHZSBTSMED,NIHZSCBTSOAM,NIHZSCCS,NIHZSHC,NIHZSIVTDLTE14,NIHZSIVTDLTE15,NIHZSIVTDLTE17,NIHZSIVTDLTE2,NIHZSLFS,NIHZSMAC,NIHZSMAC,NIHZSMAC_ACE,NIHZSMAC_CRS,NIHZSMAC_TAL,NIHZSOAM,NIHZSPHYDL,NIHZSPHYDLDCM,NIHZSPHYUL,NIHZSPHYUL,NIHZSPHYULDCM,NIHZSSOAM,NIHZSSOAM2,NIHZSWEBEM,NIHZYSCP,NIHZYSLRCSYNC,NIMASSOAM,NIMASUB,NIMHSRF_FALUCPRIA,NINASRF_CPRIA,NIOHWBBSOC,NIOHWIVCT,NIOHWRFSU,NIOSCCS,NIOSDCCS,NIOSDK,NIOSDL,NIOSDN,NIOSH2,NIOSHC,NIOSHC,NIOSHCFHS,NIOSHCFHS,NIOSHCR4,NIOSLOM,NIOSLOM,NIOSRFAS,NIOSUB,NIOSUB,NIOSUD,NIOUSSOAM,NIOYSE2,NIOYSR2,NISOPER,NISSOAM,NITSSNIDD,NITSSOAM,NITSSWEBEM,NIULSCCS,NIULSCCS,NIULSD1,NIULSLFS,NIWROMS,NIWRSBM,NIWRSRF,NIWRSRF,NIWRSSOAM,NIWRSSOAM_SCT,NIWRSWEBEM,NIWRYSNE,RABLTEEIUIROAM,RABLTEEIUMHOAM,RABLTEIVCBECP,RABLTEIVCBEOAM,RABLTEIVCBEUP1,RABLTEIVCBEUP2,RABLTEIVJLTERPHD,RABLTEIVPKRCP,RABLTEIVPKRCPPET,RABLTEIVPWROAM,RABLTESPEC,RABLTESPECAIF,RABLTESPECJOAMHD,RABLTESPECJT,RABLTESPEC_PDDB,RABLTESPEC


See the whole table with table.as_data_frame()
Top-10 Hit Ratios: 


k,hit_ratio
1,0.1
2,0.1
3,0.2
4,0.2
5,0.2
6,0.2
7,0.3
8,0.4
9,0.4
10,0.4


### Trying AutoML

In [103]:
aml = H2OAutoML(max_runtime_secs = 120)

In [105]:
aml.train(x=training_columns, y=label, training_frame=hf_df_small, leaderboard_frame = hf_df_test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [106]:
aml.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
XGBoost_1_AutoML_20190716_152537,0.0213904,4.85067,0.984721,0.969675


In [107]:
aml = H2OAutoML(max_runtime_secs = 600)

In [109]:
aml.train(x=training_columns, y=label, training_frame=hf_df_small, leaderboard_frame = hf_df_test)

AutoML progress: |███████████████████████████████████████████████████

TypeError: string indices must be integers